## Problem Statement

### Housing Price Prediction

The training dataset contains three examples with four features (size, bedrooms, floors, and age).

| Size (sqft) | Number of Bedrooms | Number of Floors | Age of Home | Price (1000s dollars) |
|-------------|--------------------|------------------|-------------|-----------------------|
| 2104        | 5                  | 1                | 45          | 460                   |
| 1416        | 3                  | 2                | 40          | 232                   |
| 852         | 2                  | 1                | 35          | 178                   |

We will build a linear regression model using these values so you can then predict the price for other houses.

In [17]:
# 4 features and 3 samples
import numpy as np

X_train = np.array([[2104.       ,    5.       ,    1.       ,   45.       ],
 [2031.57894737,    4.78947368,    1.10526316,   44.47368421],
 [1959.15789474,    4.57894737,    1.21052632,   43.94736842],
 [1886.73684211,    4.36842105,    1.31578947,   43.42105263],
 [1814.31578947,    4.15789474,    1.42105263,   42.89473684],
 [1741.89473684,    3.94736842,    1.52631579,   42.36842105],
 [1669.47368421,    3.73684211,    1.63157895,   41.84210526],
 [1597.05263158,    3.52631579,    1.73684211,   41.31578947],
 [1524.63157895,    3.31578947,    1.84210526,   40.78947368],
 [1452.21052632,    3.10526316,    1.94736842,   40.26315789],
 [1386.31578947,    2.94736842,    1.94736842,   39.73684211],
 [1326.94736842,    2.84210526,    1.84210526,   39.21052632],
 [1267.57894737,    2.73684211,    1.73684211,   38.68421053],
 [1208.21052632,    2.63157895,    1.63157895,   38.15789474],
 [1148.84210526,    2.52631579,    1.52631579,   37.63157895],
 [1089.47368421,    2.42105263,    1.42105263,   37.10526316],
 [1030.10526316,    2.31578947,    1.31578947,   36.57894737],
 [ 970.73684211,    2.21052632,    1.21052632,   36.05263158],
 [ 911.36842105,    2.10526316,    1.10526316,   35.52631579],
 [ 852.       ,    2.       ,    1.       ,   35.       ]]
)
y_train = np.array([460.        , 436.        , 412.        , 388.        ,
 364.        , 340.        , 316.        , 292.        ,
 268.        , 244.        , 229.15789474, 223.47368421,
 217.78947368, 212.10526316, 206.42105263, 200.73684211,
 195.05263158, 189.36842105, 183.68421053, 178.        ])



Feature Scaling and Feature enginerring

Explore feature engineering and polynomial regression which allows you to use the machinery of linear regression to fit very complicated, even very non-linear functions


In [18]:
def zscore_normalize_features(X):
    """
    computes  X, zcore normalized by column
    
    Args:
      X (ndarray (m,n))     : input data, m examples, n features
      
    Returns:
      X_norm (ndarray (m,n)): input normalized by column
      mu (ndarray (n,))     : mean of each feature
      sigma (ndarray (n,))  : standard deviation of each feature
    """
    # find the mean of each column/feature
    mu     = np.mean(X, axis=0)                 # mu will have shape (n,)
    # find the standard deviation of each column/feature
    std_dev  = np.std(X, axis=0)                  # sigma will have shape (n,)
    # element-wise, subtract mu for that column from each example, divide by std for that column
    X_normalized = (X - mu) / std_dev    

    return X_normalized

In [19]:
X_norm = zscore_normalize_features(X_train)
print(f"Peak to Peak range by column in Raw        X:{np.ptp(X_train,axis=0)}")   
print(f"Peak to Peak range by column in Normalized X:{np.ptp(X_norm,axis=0)}")

Peak to Peak range by column in Raw        X:[1.2520000e+03 3.0000000e+00 9.4736842e-01 1.0000000e+01]
Peak to Peak range by column in Normalized X:[3.29101537 3.25051591 3.13339781 3.29501789]


# Feature engineering is all about creating new features from existing features

In [44]:

X_train_feature_engn = np.c_[X_norm, X_norm[:,0]**2, X_norm[:,1]**2, X_norm[:,2]**2, X_norm[:,3]**2]
# these are the new features that we have created

In [20]:
def f_x_wb(x, w, b): 
    """
    single predict using linear regression
    Args:
      x (ndarray): Shape (n,) example with multiple features
      w (ndarray): Shape (n,) model parameters   
      b (scalar):             model parameter 
      
    Returns:
      p (scalar):  prediction
    """
    p = np.dot(x, w) + b     
    return p   

In [21]:
def compute_gradient(X, y, w, b): 
    """
    Computes the gradient for linear regression 
    Args:
      X (ndarray (m,n)): Data, m examples with n features
      y (ndarray (m,)) : target values
      w (ndarray (n,)) : model parameters  
      b (scalar)       : model parameter
      
    Returns:
      dj_dw (ndarray (n,)): The gradient of the cost w.r.t. the parameters w. 
      dj_db (scalar):       The gradient of the cost w.r.t. the parameter b. 
    """
    m,n = X.shape           #(number of examples, number of features)
    dj_dw = np.zeros(n)
    dj_db = 0

    for i in range(m):                             
        
        err = f_x_wb(X[i],w,b) - y[i]  # error for the i-th example

        dj_db += err
        dj_dw += (err * X[i])
                           
        
    return dj_db/m , dj_dw/m


In [22]:
def train(X, y, w_in, b_in, gradient_function, alpha, num_iters): 
    """
    Performs batch gradient descent to learn w and b. Updates w and b by taking 
    num_iters gradient steps with learning rate alpha
    
    Args:
      X (ndarray (m,n))   : Data, m examples with n features
      y (ndarray (m,))    : target values
      w_in (ndarray (n,)) : initial model parameters  
      b_in (scalar)       : initial model parameter
      gradient_function   : function to compute the gradient
      alpha (float)       : Learning rate
      num_iters (int)     : number of iterations to run gradient descent
      
    Returns:
      w (ndarray (n,)) : Updated values of parameters 
      b (scalar)       : Updated value of parameter 
      """
    
    w = w_in 
    b = b_in

    for i in range(num_iters):

        # Calculate the gradient and update the parameters
        dj_db,dj_dw = gradient_function(X, y, w, b)   

        # Update Parameters using w, b, alpha and gradient
        w = w - alpha * dj_dw               
        b = b - alpha * dj_db               
              
    return w, b #return final w,b

In [50]:
# initial_w = np.zeros(len(X_train[0]))
initial_w = np.zeros(len(X_train_feature_engn[0]))
initial_b = 0

iterations = 100000
alpha = 1.0e-4

w_final, b_final = train(X_train_feature_engn, y_train, initial_w, initial_b, compute_gradient, alpha, iterations)
print(f"b,w found by gradient descent: {b_final:0.2f} , {w_final} ")

# w_final, b_final = train(X_norm, y_train, initial_w, initial_b, compute_gradient, alpha, iterations)
# print(f"b,w found by gradient descent: {b_final:0.2f} , {w_final} ")

# w_final, b_final = train(X_train, y_train, initial_w, initial_b, compute_gradient,alpha, iterations)
# print(f"b,w found by gradient descent: {b_final:0.2f} , {w_final} ")

b,w found by gradient descent: 176.77 , [ 26.22672059  15.824422    87.55045871  30.57785328  45.32979114
  33.02557268 -29.07332073  50.06504874] 


1) Z-Norm
iterations = 100000
alpha = 1.0e-4
b,w found by gradient descent: 277.78 , [ 28.34313317  30.29060682 -19.94524968  27.39362005] 

CHECKING THE ACCURACY


In [51]:
m = X_train.shape[0]

for i in range(m):
    # print(f"Prediction: {f_x_wb(X_train[i],w_final,b_final):0.2f}, target value: {y_train[i]}")
    # print(f"Prediction: {f_x_wb(X_norm[i],w_final,b_final):0.2f}, target value: {y_train[i]}")
    print(f"Prediction: {f_x_wb(X_train_feature_engn[i],w_final,b_final):0.2f}, target value: {y_train[i]}")

Prediction: 480.95, target value: 460.0
Prediction: 443.90, target value: 436.0
Prediction: 409.52, target value: 412.0
Prediction: 377.84, target value: 388.0
Prediction: 348.83, target value: 364.0
Prediction: 322.52, target value: 340.0
Prediction: 298.89, target value: 316.0
Prediction: 277.94, target value: 292.0
Prediction: 259.68, target value: 268.0
Prediction: 244.11, target value: 244.0
Prediction: 235.67, target value: 229.15789474
Prediction: 231.62, target value: 223.47368421
Prediction: 226.60, target value: 217.78947368
Prediction: 220.61, target value: 212.10526316
Prediction: 213.65, target value: 206.42105263
Prediction: 205.73, target value: 200.73684211
Prediction: 196.83, target value: 195.05263158
Prediction: 186.96, target value: 189.36842105
Prediction: 176.13, target value: 183.68421053
Prediction: 164.32, target value: 178.0
